# Lab 8 - Evidence-based Prompt Iteration

在不改变基座模型的前提下，通过提示词可以实质性提高模型的表现。

因为提示词是自然语言构建的，因此提示词给了人们一个通过自然语言编程来生产app的机会。在我看来这是革命性的进展。

在上一讲中，我们已经介绍了如何编写第一个user prompt，这只是cold start，更多的工作是要在后续的工作中不断发现bad case，并且通过提示词去修复。这是目前我所了解的，最靠谱的提示词迭代方式。

# 1. 项目背景

## 1.1 业务介绍
一个服务团队要检查其员工是否通知学生补课。下述两个条件是他们的“质检规则”

    - 通过企业微信发送了补课消息，就算通知
    - 进行了电话外呼（即使没有接通），也算通知

员工可能在当天进行了上课提醒，这不是补课通知。


## 1.2数据集
`classification_sample.csv`有3列，第一列是`text`，代表企微聊天记录；第二列是`callout`，代表外呼次数；第三列是`flag`，代表业务团队专家判断的是否通过了“质检”。

## 1.3 任务
利用提示词，训练一个模型，能够自动判断员工是否通过了“质检”。

# 2. LLM Classifier
## 2.1 Baseline

In [8]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()  # 敏感信息不能明文展示
api_key = os.getenv('OPENAI_API_KEY')

from openai import OpenAI

client = OpenAI(
            api_key=api_key,
            base_url="https://api.aaaapi.com/v1")

In [3]:
data = pd.read_csv("../data/classification_sample.csv")

In [12]:
def classify_with_llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt}
        ],
        response_format={"type": "json_object"}  # 返回json格式，以免解析问题
    )
    return response.choices[0].message.content

In [10]:
def _construct_prompt(text, callout):
    return f"""
    ```
    你在检查LA是否向用户发起了补课提醒。
    如果沟通记录中包含补课消息，则flag为1；否则为0。
    你要检查的沟通记录是
    {text}

    对该用户的外呼次数是{callout}。如果外呼次数大于1，则flag为1，否则为0

    你的输出结果是一个json{{"flag": 0/1}}
    ```
    """

In [14]:
# 创建一个空列表来存储预测结果
predicted_flags = []

# 遍历数据集并获取预测

for index, row in data.iterrows():
    flag = classify_with_llm(_construct_prompt(row["text"], row["callout"]))
    predicted_flags.append(flag)

# 将预测结果添加到数据集中
data["predicted_flag"] = predicted_flags



In [ ]:
from sklearn.metrics import classification_report
import json

# 将json字符串转换为数值
data['predicted_flag_numeric'] = data['predicted_flag'].apply(lambda x: json.loads(x)['flag'])

# 计算F1分数
perf = classification_report(data['flag'], data['predicted_flag_numeric'])
print(perf)


# 2.2 迭代

找到flag=1但是predicted_flag=0的case，或者flag=0但是predicted_flag=1的case。

分析这些case，看如何优化你的提示词。

在优化时，最好遵循这样的实践：

    1. 首先验证修改后的提示词是否正确解决这个case
    2. 重跑所有case，看看其他case是否有变化。如果其他case的效果恶化严重，那么需要放弃这个修改


考虑你可以使用的工具：

    - 使用KIMI 模版来结构化输出业务判定规则，具体说明什么叫补课
    - 提供一些case来向大模型“解释”你的判定规则


